<a href="https://colab.research.google.com/github/unihong/MS-AI-School/blob/main/20230731_Pytorch_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision.datasets import VOCSegmentation
from torch.utils.data import Dataset
import cv2
import os

class customVOCSegmentation(VOCSegmentation) :
    def __init__(self, root, mode="train", download=True,transforms=None):
        self.root = root
        super().__init__(root=self.root, image_set=mode,
                         download=self.check_if_path_exists(), transforms=transforms )
        # VOCSegmentation.__init__()
        # VOCSegmentation 클래스의 생성자를 호출하며, 변경이 필요한 인자에 대해 받아서 넘겨줌.
        # 해당 생성자에서 Custom Dataset의 생성자에서 필요한 작동을 모두 정의하고 있음
        # (한 줄로 끝내서 간편함)
        # 즉, 데이터셋 자체와 라벨 이미지 모두가 이 단락에서 생성이 완료됨

        # self.check_if_path_exists() 편의 함수는 download 됐는지에 대한 여부 판별
        #


    def __getitem__(self, idx) :
        img = cv2.imread(self.images[idx])
        mask = cv2.imread(self.masks[idx])

        # 부모 클래서에서 원본 이미지는 self.images라는 이름으로 호출하도록 정의되어 있음
        # 라벨에 해당하는 마스크는 self.masks라는 이름으로 호출하도록 정의되어 있음
        # 해당 리스트는 이미지 자체가 아닌 해당 이미지 파일 경로만을 담고 있기 때문에, imread로 읽어줘야 함
        if self.transforms: # == if self.transrform is not None :
            augmented = self.transforms(image=img, mask=mask)
            # cv2fmf chleogks 사용하는 것으로 가정하였으므로, torchvision의 transform 이 아닌
            # albumentations 모듈을 사용하는 형태로 작성

            # 또한, segmentation 의 경우 label 역할을 하는 mask 역시 이미지이며,
            # image와 한 쌍을 이루기 때문에 transform 과정에서 함께 동일한 augmentation이 진행되어야 함.
            img = augmented['image']
            mask = augmented['mask']


        return img, mask

    def check_if_path_exists(self) :
        #return False if os.path.exists(self.root) else True
        return not os.path.exists(self.root)
        # self.root 에 해당하는 data 폴더는 원래는 만들어지지 않았다가
        # 클래스 선언과 함께 다운로드 받으며 만들어질 것
        # 따라서, 해당 폴더가 존재한다면 다운로드 받을 필요가 없음 (False 반환)
        # 반대로, 해당 폴더가 없다면 다운로드 받아야 함.(True 반환)

    # __len__ 의 경우, 부모 클래스에서 자신의 구조를 정의하면서 함께 정의를 한 상태이기 때문에
    # 주가로 재정의할 필요없음

if __name__ == "__main__" :
    # VOCSegmentation()
    dataset = customVOCSegmentation("./data")
    for item in dataset:
        img, mask = item
        summary = cv2.copyTo(img, mask)
        # copyTo 함수는 mask의 0이 아닌 부분에 img를 복사하도록 함 -> 라벨링 된 영역만 이미지가 표시됨
        marked = cv2.addWeighted(img, 0.5, mask, 0.5, 0)
        # addWeight 함수는 두 이미지에 대해서 입력한 가중치만금 더함. 원본이미지에 mask가 겹쳐 나옴.
        # cv2.imshow("org", img)
        # cv2.imshow("mask", mask)
        #cv2.imshow("summary", summary)
        cv2.imshow("marked", marked)
        cv2.waitKey()
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') :
            break

In [18]:
import torch
import numpy as np
import time
import os
from tqdm import tqdm
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models.segmentation import deeplabv3_resnet101
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from argparse import ArgumentParser
from torchvision.datasets import VOCSegmentation

class SegLearner:
    def __init__(self, model, optimizer, criterion, train_dataloader, valid_dataloader):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.criterion = criterion.to(self.device)
        self.optimizer = optimizer
        # 모델, 손실함수, 옵티마이저

        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        # 데이터로더

        #self.args = args
        # 터미널 인자로 받은 필요값들

        self.start_epoch = 0
        self.metrics = {
            "train_loss": [],
            "train_acc": [],
            "train_miou": [],
            "val_loss": [],
            "val_acc": [],
            "val_miou": []
        }
        # resume이 걸릴 경우 / 학습이 저장될 경우 필요한 값들

    def train(self):
        for epoch in range(self.start_epoch, 10):
            self.model.train()
            train_loss = 0.0
            train_corr = 0.0
            val_loss = 0.0
            val_corr = 0.0

            train_iou = 0.0
            val_iou = 0.0
            # mIoU 계산을 위한 IoU 총합치 저장 변수

            for i, (inputs, labels) in enumerate(tqdm(self.train_dataloader)):
                inputs = inputs.float().to(self.device)
                labels = labels.long().to(self.device)

                self.optimizer.zero_grad() # 가중치 업데이트를 위한 optimizer 초기화

                outputs = self.model(inputs) # 순전파
                outputs = outputs["out"] # deeplab은 output이 dict 형태로 쓸 수 있도록 나오므로, 출력치 key로 받아옴

                loss = self.criterion(outputs, labels)
                loss.backward() # 역전파
                self.optimizer.step() # 가중치 업데이트

                preds = torch.argmax(outputs, dim=1) # output으로부터 class에 대한 예측값을 얻음

                train_loss += loss.item()
                corrects = torch.sum(preds == labels.data) # labels와 preds는 이미지 형태
                # 즉, 위의 line은 label 이미지와 preds 이미지를 겹쳤을 때, 일치하는 픽셀의 갯수가 나옴

                # 이미지 형태이므로, 여기에서 나오는 총 길이는 최대 520x520 크기일것 (pretrained 기준)
                batch_size = inputs.size(0)
                train_corr += corrects.double() / (batch_size * 520 * 520)
                # Pixel accuracy를 이용한 정확도 계산

                train_iou += self.calc_iou(preds, labels.long())

            _t_loss = train_loss / len(self.train_dataloader)
            # 이번 epoch의 평균 train loss
            _t_acc = train_corr / len(self.train_dataloader.dataset)
            # 이번 epoch의 평균 pixel accuracy
            _t_iou = train_iou / len(self.train_dataloader.dataset)
            # 이번 epoch의 miou

            self.metrics["train_loss"].append(_t_loss)
            self.metrics["train_acc"].append(_t_acc)
            self.metrics["train_miou"].append(_t_iou)

            print(f"[{epoch + 1} / 10 ] train loss : {_t_loss}",
                  f"train acc : {_t_acc}, train mIoU : {_t_iou}")

            # validation 시작
            self.model.eval()
            with torch.no_grad():
                for val_i, (inputs, labels) in enumerate(tqdm(self.valid_dataloader)):
                    inputs = inputs.to(self.device)
                    labels = labels.long().to(self.device)

                    outputs = self.model(inputs)
                    outputs = outputs["out"]
                    loss = self.criterion(outputs, labels)
                    preds = torch.argmax(outputs, dim=1)

                    val_loss += loss.item()
                    corrects = torch.sum(preds == labels.data)

                    batch_size = inputs.size(0)
                    val_corr += corrects.double() / (batch_size * 520 * 520)
                    # Pixel accuracy를 이용한 정확도 계산
                    val_iou += self.calc_iou(preds, labels.data)

            _v_loss = val_loss / len(self.valid_dataloader)
            _v_acc = val_corr / len(self.valid_dataloader.dataset)
            _v_miou = val_iou / len(self.valid_dataloader.dataset)

            self.metrics["val_loss"].append(_v_loss)
            self.metrics["val_acc"].append(_v_acc)
            self.metrics["val_miou"].append(_v_miou)

            print(f"[{epoch + 1} / 10 ] valid loss : {_v_loss}",
                  f"valid acc : {_v_acc}, valid mIoU : {_v_miou}")

            self.save_ckpts(epoch)


    def load_ckpts(self):
        '''
        path: .pt 파일이 저장된 위치
        '''
        ckpt_path = os.path.join("./weight", "weight.pt")
        # 터미널 인자 args로부터 지정된 weight 로딩 경로를 받아옴

        ckpt = torch.load(ckpt_path) # .pt 파일을 불러와서 dictionary 형태로 선언
        self.model.load_state_dict(ckpt["model"]) # dict 안에 있는 "model"키로 저장할 모델 가중치 로드
        self.optimizer.load_state_dict(ckpt["optimizer"])
        self.start_epoch = ckpt["epoch"]
        self.metrics = ckpt["metrics"]

    def save_ckpts(self, epoch, file_name=None):
        # 체크포인트 저장을 처리하기 위한 편의함수
        if not os.path.exists("./weight"):
            os.makedirs("./weight", exist_ok=True)
        # 모델 가중치가 저장될 폴더가 없을 경우, 오류가 날 수 있으므로
        # 터미널 인자 args에서 받은 model_folder_path가 있는지 확인 후, 없으면 생성

        if file_name is None:
            to_save_path = os.path.join("./weight", "weight.pt")
        else:
            to_save_path = os.path.join("./weight", file_name)
        # file name 커스텀을 위한 조건식 부분

        torch.save(
            {
                "model": self.model.state_dict(), # 현재 가중치 값
                "optimizer": self.optimizer.state_dict(), # optimizer의 현재 수치
                "epoch": epoch,
                "metrics": self.metrics
            }, to_save_path
        )


    @staticmethod
    def calc_iou(preds, labels):
        total_iou = 0.0
        # 들어온 batch의 IoU 총합
        for inp, ans in zip(preds, labels):
            # inp == preds에서 들어온, 예측치가 담겨있는 단일 이미지(텐서)
            # ans == labels에서 들어온, 정답치가 담겨있는 단일 이미지(텐서)
            inp = inp.cpu().numpy()
            # inp는 device에 넘어가있는 텐서이므로, cpu로 넘겨준 뒤 텐서에서 numpy 행렬로 변환
            ans = ans.cpu().numpy()

            union_section = np.logical_or(inp, ans)
            inter_section = np.logical_and(inp, ans)
            # 위에서 or, and 연산을 통해 얻은 numpy 행렬은 Boolean 행렬이며, 계산에 사용할 수 있는 수가 아님

            # cv2.imshow("union", union_section.astype(np.uint8) * 255)
            # 마스크 확인용 imshow 코드

            uni_sum = np.sum(union_section)
            inter_sum = np.sum(inter_section)
            # 해당하는 행렬의 총 픽셀 수 (T/F Boolean mask 형태로 나올 것이므로 sum을 하면 픽셀 수를 얻음)
            # == 해당 영역의 넓이
            if uni_sum != 0:
                total_iou += inter_sum / uni_sum
            else:
                total_iou += 0
            # 성공적인 데이터 입력의 경우 uni_sum이 0이 될 일은 없음 (정답지 mask가 완전히 검은 이미지는 없으므로)
            # 그러나 만약의 경우를 대비하여 0으로 나눈 오류가 나지 않도록 처리

            # 교집합 넓이 / 합집합 넓이 = IoU

        return total_iou





if __name__=="__main__":
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # 실험에 의해 도출된, 가장 좋은 결과를 도출하는 평균/표준편차 값들

    train_transforms = A.Compose([
        A.Resize(520, 520), # DeepLabV3 pretrain 모델이 사용할 이미지 크기 (520x520)

        A.HorizontalFlip(p=0.5), # 50% 확률로 좌우반전

        A.Normalize(mean=mean, std=std),
        ToTensorV2() # torch 모델에 넣을 수 있도록 ToTensor 사용
    ])

    valid_transforms = A.Compose([
        A.Resize(520, 520), # DeepLabV3 pretrain 모델이 사용할 이미지 크기 (520x520)
        A.Normalize(mean=mean, std=std),
        ToTensorV2()
    ])

    # # 아래에 dataset과 dataloader 등등에 필요한 터미널 인자를 지정하기 위해 argparser 사용
    # parser = ArgumentParser()
    # parser.add_argument("--data_path", type=str, default="./data",
    #                     help="데이터셋 파일이 저장되거나 로딩될 지점")
    # parser.add_argument("--weight_folder_path", type=str, default="./weight",
    #                     help="가중치가 저장될 폴더의 경로")
    # parser.add_argument("--weight_file_name", type=str, default="weight.pt",
    #                     help="저장될 가중치 파일의 이름")
    # parser.add_argument("--batch_size", type=int, default=2,
    #                     help="Batch size")
    # parser.add_argument("--num_workers", type=int, default=4,
    #                     help="데이터로더가 사용할 프로세스 수, 사용자 컴퓨터 환경에 따라 달라질 수 있음")
    # parser.add_argument("--learning_rate", type=float, default=0.001,
    #                     help="learning rate")
    # parser.add_argument("--weight_decay", type=float, default=0.01,
    #                     help="lr의 감소율")
    # parser.add_argument("--resume", action="store_true",
    #                     help="학습 재개 여부, store_true가 지정되면 터미널 인자로 선언되어야 true가 들어옴")
    # parser.add_argument("--epochs", type=int, default=10,
    #                     help="이번 학습의 총 epoch 수")
    # args = parser.parse_args()

    train_dataset = customVOCSegmentation("./data", mode="train", transforms=train_transforms)
    valid_dataset = customVOCSegmentation("./data", mode="val", transforms=valid_transforms)
    # dataset 선언

    train_dataloader = DataLoader(train_dataset, batch_size=2, num_workers=4,
                                  shuffle=True, drop_last=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=2, num_workers=4)
    # dataloader 선언

    model = deeplabv3_resnet101(pretrained=True)
    # model 선언 (pretrained 모델 사용)

    optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.01)
    criterion = CrossEntropyLoss(ignore_index=255)

    # train을 위해 만든 편의 class 선언
    learner = SegLearner(model, optimizer, criterion, train_dataloader, valid_dataloader)

    # if args.resume:
    #     learner.load_ckpts()

    # 학습 시작
    learner.train()


100%|██████████| 732/732 [04:10<00:00,  2.92it/s]


[1 / 10 ] train loss : 1.2704386176551625 train acc : 0.37406201372384973, train mIoU : 0.0032783750210137624


100%|██████████| 725/725 [01:26<00:00,  8.34it/s]


[1 / 10 ] valid loss : 1.2770283775288482 valid acc : 0.3695806942964129, valid mIoU : 0.024406891690545315


100%|██████████| 732/732 [04:07<00:00,  2.95it/s]


...

[9 / 10 ] valid loss : 1.2137249261962957 valid acc : 0.3739876485925817, valid mIoU : 0.0


100%|██████████| 732/732 [04:07<00:00,  2.96it/s]


[10 / 10 ] train loss : 1.1839683448763494 train acc : 0.3745176645811105, train mIoU : 0.0


100%|██████████| 725/725 [01:26<00:00,  8.39it/s]


[10 / 10 ] valid loss : 1.170155117059576 valid acc : 0.3739876485925817, valid mIoU : 0.0
